This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks import *


In [ ]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    "embedding_corruption_ratio" : 0.1, 
    "partial_embedding_reconstruction" : True
}

generator_config = {
    'num_clusters' : 7
}

pretext_tasks = [CorruptedEmbeddingsReconstruction]

NameError: name 'CorruptedEmbeddingsReconstruction' is not defined

In [ ]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [4]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_tasks=pretext_tasks)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


([2.0760700702667236,
  2.0044093132019043,
  1.987534523010254,
  1.9458147287368774,
  1.9269682168960571,
  1.9112441539764404,
  1.8772671222686768,
  1.8513157367706299,
  1.8210457563400269,
  1.7912354469299316,
  1.7470089197158813,
  1.6758590936660767,
  1.5912013053894043,
  1.5964360237121582,
  1.4586825370788574,
  1.4656621217727661,
  1.411342978477478,
  1.3999435901641846,
  1.2984179258346558,
  1.2486016750335693,
  1.137729525566101,
  1.0697301626205444,
  1.0467299222946167,
  0.9827516674995422,
  1.006561517715454,
  1.1194157600402832,
  0.850692093372345,
  0.8762112855911255,
  0.9030970335006714,
  0.7201496362686157,
  0.7467393279075623,
  0.8658332228660583,
  0.8404584527015686,
  0.7336710095405579,
  0.7520580887794495,
  0.5995291471481323,
  0.7010443210601807,
  0.6454289555549622,
  0.5717145800590515,
  0.662835419178009,
  0.5634177923202515,
  0.6238008141517639,
  0.41033297777175903,
  0.4414980411529541,
  0.5541046857833862,
  0.50838935375